✅ Step 1: Load Your Classifier and Make a Prediction

In [ ]:
import pickle
import numpy as np

# Load trained model
with open("rf_classifier.pkl", "rb") as f:
    classifier = pickle.load(f)

# Sample patient data (replace with actual input)
patient_data = {
    "age": 57,
    "sex": 1,
    "cp": 2,
    "trestbps": 140,
    "chol": 250,
    "fbs": 0,
    "restecg": 1,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 1.4,
    "slope": 2,
    "ca": 0,
    "thal": 2
}

# Ensure feature order matches training
features = np.array([list(patient_data.values())]).reshape(1, -1)

# Get prediction
prediction = classifier.predict(features)[0]  # 0 = No Heart Disease, 1 = Likely Heart Disease
prediction

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


np.int64(1)

In [ ]:
!pip install transformers accelerate -q

In [2]:
!pip install sacremoses -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.6 MB/s eta 0:00:00


from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Conversational Model
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
mistral_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")
mistral_pipe = pipeline("text-generation", model=mistral_model, tokenizer=mistral_tokenizer)

# Biomedical Model
bio_tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
bio_model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT-Large", device_map="auto")
bio_pipe = pipeline("text-generation", model=bio_model, tokenizer=bio_tokenizer)


In [15]:
def route_query(query: str) -> str:
    biomedical_keywords = ["what is", "define", "symptoms", "treatment", "diagnosis", "medicine", "drug", "clinical", "risk factor"]

    if any(kw in query.lower() for kw in biomedical_keywords):
        return "biomedical"
    return "conversational"

In [18]:
def ask_heart_bot(query: str):
    route = route_query(query)

    if route == "biomedical":
        response = bio_pipe(query, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
    else:
        # Proper Mistral-style prompt formatting
        prompt = f"[INST] {query} [/INST]"
        response = mistral_pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]['generated_text']

    return response.strip()


In [19]:
import re

def clean_response(response: str):
    # Remove any non-standard tokens or unwanted text
    response = re.sub(r"<.*?>", "", response)
    return response.strip()


In [20]:
def ask_heart_bot(query: str):
    route = route_query(query)

    if route == "biomedical":
        response = bio_pipe(query, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
    else:
        prompt = f"[INST] {query} [/INST]"
        response = mistral_pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]['generated_text']

    return clean_response(response)


In [24]:
def ask_heart_bot(query: str):
    route = route_query(query)

    # Set parameters to handle longer responses and adjust temperature for varied output
    max_tokens = 600
    temperature = 0.7
    pad_token_id = 2  # Ensure that padding is handled correctly

    # Check the route and use the appropriate pipeline
    if route == "biomedical":
        # Generating a response from the biomedical model
        response = bio_pipe(query, max_new_tokens=max_tokens, do_sample=True, temperature=temperature, pad_token_id=pad_token_id)[0]['generated_text']
    else:
        # Generating a response from the other model
        prompt = f"[INST] {query} [/INST]"
        response = mistral_pipe(prompt, max_new_tokens=max_tokens, do_sample=True, temperature=temperature, pad_token_id=pad_token_id)[0]['generated_text']

    # Clean and return the response
    return clean_response(response)

In [25]:
print(ask_heart_bot("I am having chest pains. What could be the cause?"))
print(ask_heart_bot("What is myocardial infarction?"))
print(ask_heart_bot("Suggest a diet plan for a heart patient."))


[INST] I am having chest pains. What could be the cause? [/INST] I'm an AI language model and cannot physically examine you or diagnose conditions. Chest pain can be caused by a variety of conditions, some of which are serious and require immediate medical attention. Common causes of chest pain include:

1. Angina: This is a type of chest pain caused by reduced blood flow to the heart muscle due to narrowed or blocked coronary arteries. Angina is usually described as a feeling of pressure or tightness in the chest, often accompanied by shortness of breath, nausea, and fatigue.
2. Heart attack: A heart attack occurs when the blood flow to a part of the heart is blocked, leading to damage or death of heart muscle. Chest pain during a heart attack is often described as a heavy, crushing sensation that may spread to the neck, jaw, back, or arms.
3. Pulmonary embolism: This is a blockage in a pulmonary artery, usually caused by a blood clot. Chest pain from a pulmonary embolism is often des

In [35]:
# light_heart_pipeline.py

class UnifiedHeartHealthBotPipeline:
    def __init__(self, route_query, clean_response):
        self.route_query = route_query
        self.clean_response = clean_response

    def ask_bot(self, query, bio_pipe, mistral_pipe):
        route = self.route_query(query)

        if route == "biomedical":
            response = bio_pipe(query, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
        else:
            prompt = f"[INST] {query} [/INST]"
            response = mistral_pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]['generated_text']

        return self.clean_response(response)


In [38]:
import pickle
import re

# Routing logic
def route_query(query: str) -> str:
    biomedical_keywords = ["what is", "define", "symptoms", "treatment", "diagnosis", "medicine", "drug", "clinical", "risk factor"]
    if any(kw in query.lower() for kw in biomedical_keywords):
        return "biomedical"
    return "conversational"

# Cleanup logic
def clean_response(response: str):
    return re.sub(r"<.*?>", "", response).strip()

# Lightweight pipeline with lazy model loading
class UnifiedHeartHealthBotPipeline:
    def __init__(self, route_query, clean_response):
        self.route_query = route_query
        self.clean_response = clean_response
        self._bio_pipe = None
        self._mistral_pipe = None

    def _load_bio_pipe(self):
        if self._bio_pipe is None:
            from transformers import pipeline
            self._bio_pipe = pipeline("text-generation", model="microsoft/BioGPT-Large", tokenizer="microsoft/BioGPT-Large", device_map="auto")
        return self._bio_pipe

    def _load_mistral_pipe(self):
        if self._mistral_pipe is None:
            from transformers import pipeline
            self._mistral_pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", tokenizer="mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")
        return self._mistral_pipe

    def ask_bot(self, query):
        route = self.route_query(query)
        if route == "biomedical":
            pipe = self._load_bio_pipe()
            response = pipe(query, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
        else:
            pipe = self._load_mistral_pipe()
            prompt = f"[INST] {query} [/INST]"
            response = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]['generated_text']
        return self.clean_response(response)

# Create instance and save
pipeline = UnifiedHeartHealthBotPipeline(route_query, clean_response)
with open("light_heart_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)


In [39]:
import pickle

# Load the pickled pipeline
with open("light_heart_pipeline.pkl", "rb") as f:
    pipeline = pickle.load(f)


In [40]:
response2 = pipeline.ask_bot("How are you today?")
print("Conversational Response:\n", response2)


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Conversational Response:
 [INST] How are you today? [/INST] I'm an artificial intelligence and don't have feelings or emotions. I'm here to help answer your questions to the best of my ability. Is there a specific question you have in mind?


In [41]:
response1 = pipeline.ask_bot("What is myocardial infarction?")
print("Biomedical Response:\n", response1)

Device set to use cpu


Biomedical Response:
 What is myocardial infarction?   ▃


In [42]:
test_queries = [
    "What are the symptoms of a heart attack?",                      # Biomedical
    "Define arrhythmia.",                                           # Biomedical
    "How can high blood pressure be managed?",                      # Biomedical
    "Tell me a joke about doctors.",                                # Conversational
    "Can you give lifestyle tips for a healthy heart?",             # Conversational
    "What medications are used for cholesterol control?",           # Biomedical
    "How's the weather in your circuits today?",                    # Conversational
    "Explain the treatment of atrial fibrillation.",                # Biomedical
    "Write a poem about staying healthy.",                          # Conversational
    "Can you suggest a heart-healthy Indian diet?"                  # Biomedical
]

# Run all queries through the unified bot
for i, q in enumerate(test_queries):
    print(f"\n🟦 Query {i+1}: {q}")
    try:
        response = pipeline.ask_bot(q)
        print(f"🟩 Response:\n{response}")
    except Exception as e:
        print(f"🟥 Error: {e}")



🟦 Query 1: What are the symptoms of a heart attack?
🟩 Response:
What are the symptoms of a heart attack?   ▃

🟦 Query 2: Define arrhythmia.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟩 Response:
Define arrhythmia.   ▃

🟦 Query 3: How can high blood pressure be managed?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟩 Response:
[INST] How can high blood pressure be managed? [/INST] High blood pressure, also known as hypertension, can be managed through several lifestyle changes and, in some cases, medication. Here are some ways to help manage high blood pressure:

1. Maintain a healthy weight: Being overweight can increase your risk of developing high blood pressure. Aim for a healthy weight by eating a balanced diet and getting regular physical activity.

2. Eat a healthy diet: A diet rich in fruits, vegetables, whole grains, and lean proteins can help lower your blood pressure. Limit your intake of sodium, processed foods, and foods high in saturated and trans fats.

3. Exercise regularly: Regular physical activity can help lower your

🟦 Query 4: Tell me a joke about doctors.


...

🟩 Response:
[INST] What medications are used for cholesterol control? [/INST] There are several types of medications that can be used to help control cholesterol levels in the blood. The specific medication prescribed depends on the individual's cholesterol levels, underlying causes, and overall health. Here are some common types of cholesterol-lowering medications:

1. Statins: These are the most commonly prescribed cholesterol-lowering drugs. They work by inhibiting an enzyme in the liver that is needed to produce cholesterol. Examples include atorvastatin (Lipidor), simvastatin (Zocor), pravastatin (Pravachol), and rosuvastatin (Crestor).

🟦 Query 7: How's the weather in your circuits today?
🟩 Response:
[INST] How's the weather in your circuits today? [/INST] I don't have a physical form or weather conditions. I'm a text-based artificial intelligence designed to process and generate text. I can't experience weather or have circuits that can be affected by it.

🟦 Query 8: Explain the

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟩 Response:
Explain the treatment of atrial fibrillation.   ▃

🟦 Query 9: Write a poem about staying healthy.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟩 Response:
[INST] Write a poem about staying healthy. [/INST] In the dance of life, where days blend and twirl,

Staying healthy is a precious pearl.
A vibrant force, a radiant charm,
A shield against life's stormy alarm.

Eat the rainbow, in every hue,
Fruits and veggies, fresh, ripe, and true.
Nourish your body, a sacred temple,
With nature's gifts, in a nurturing helm.

Hydrate well, with clear and pure,
A glassful serenade, a soothing lure.
Quench your thirst, to keep your power,
A lifeline, a fount

🟦 Query 10: Can you suggest a heart-healthy Indian diet?
🟩 Response:
[INST] Can you suggest a heart-healthy Indian diet? [/INST] Absolutely! A heart-healthy Indian diet can be rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. Here are some suggestions for creating a heart-healthy Indian meal plan:

1. Whole Grains: Incorporate plenty of whole grains into your diet such as brown rice, whole wheat roti (flatbread), quinoa, and millets. These provide fiber, B vit